---

## 📊 ETAPA 1: Compreensão do Problema

### 1.1 Contexto de Saúde Pública

**Por que este problema é importante?**

A obesidade é considerada pela OMS uma **epidemia global**:
- Afeta mais de 650 milhões de adultos mundialmente
- Está associada a diabetes tipo 2, doenças cardiovasculares e câncer
- Gera custos significativos para sistemas de saúde

**O que queremos resolver?**

Desenvolver um sistema que:
1. **Classifique** indivíduos em 7 níveis de obesidade
2. **Identifique** fatores de risco modificáveis
3. **Auxilie** profissionais de saúde em intervenções preventivas

**Como isso ajuda na prática?**

- Triagem rápida de pacientes em risco
- Personalização de intervenções (dieta, exercício, acompanhamento)
- Identificação precoce de tendências populacionais

### 1.2 Estrutura dos Dados

**Dataset:** 2.111 registros de indivíduos de México, Peru e Colômbia

**Variáveis Disponíveis:**

| Categoria | Variáveis | Por que são importantes? |
|-----------|-----------|-------------------------|
| **Demográficas** | Gender, Age, Height, Weight | Base para cálculo do IMC, diferenças metabólicas |
| **Genéticas** | family_history | Hereditariedade: 40-70% da variação do IMC |
| **Alimentação** | FAVC, FCVC, NCP, CAEC | Comportamentos modificáveis via intervenção |
| **Estilo de Vida** | FAF, TUE, CALC, SMOKE | Fatores de risco conhecidos |
| **Mobilidade** | MTRANS | Indicador indireto de atividade física |
| **Autocuidado** | SCC, CH2O | Consciência sobre saúde |

**Variável Alvo (Target):**
- `Obesity`: 7 classes ordenadas do peso insuficiente à obesidade tipo III
- Classificação baseada no IMC (Índice de Massa Corporal)

**Por que isso é um problema de classificação multiclasse?**
- Temos múltiplas categorias ordenadas (não apenas "obeso" ou "não obeso")
- Cada nível requer intervenções diferentes
- A progressão entre níveis é clinicamente relevante

---

## 🔍 ETAPA 2: Análise Exploratória de Dados (EDA)

A EDA foi usada para validar a qualidade dos dados, entender distribuições, identificar possíveis problemas (duplicatas, outliers, desbalanceamento) e levantar hipóteses sobre os principais fatores associados à obesidade. Abaixo estão apenas os achados que de fato influenciaram decisões de modelagem e de negócio.

### 2.2 Descoberta 1: Qualidade dos Dados

**O que encontramos:**
- ✅ Zero valores faltantes
- ⚠️ Registros duplicados presentes (mas esperados - veja explicação abaixo)
- ✅ Tipos de dados consistentes

**Sobre as Duplicatas:**

Encontramos alguns registros com valores idênticos em todas as 17 variáveis. **Isso NÃO é um problema** porque:

1. **Ausência de identificadores únicos**: Dataset não contém nome, CPF ou ID
2. **Coincidências são possíveis**: Em população de 2.111 pessoas, é estatisticamente plausível que algumas tenham:
   - Mesma idade, gênero, peso, altura
   - Mesmos hábitos alimentares e de atividade física
   - Mesmo meio de transporte e consumo de água
3. **Não são erros de coleta**: São indivíduos diferentes com características idênticas

**Analogia:** É como encontrar 2 pessoas que:
- Têm 25 anos
- Pesam 70kg
- Medem 1.70m  
- Não fazem exercícios
- Comem fast food frequentemente

São pessoas diferentes, mas com perfis comportamentais e físicos muito similares.

**Decisão Tomada:**
Manter todas as duplicatas porque representam observações válidas de indivíduos distintos.

**Por que isso é importante:**
- Não precisamos de técnicas de imputação (zero valores faltantes)
- Duplicatas não comprometem a modelagem (são casos reais)
- Permite focar na análise em vez de limpeza excessiva

**Implicação Prática:**
Dataset pronto para modelagem direta, mas ainda precisamos validar distribuições e outliers.

### 2.3 Descoberta 2: Distribuição das Classes de Obesidade

**O que analisamos:**
Frequência de cada nível de obesidade no dataset.

**Principais Achados:**
- Classes mais frequentes: Obesity Type I e Overweight Level II
- Classes menos frequentes: Insufficient Weight e Normal Weight
- Razão max/min entre classes: ~3x (desbalanceamento moderado)

**Por que isso importa:**
- Modelos podem ter **viés** para classes majoritárias
- Precisamos de métricas além de acurácia (F1-score, recall por classe)
- Pode ser necessário balanceamento (SMOTE, class weights)

**Decisão Tomada:**
Usar **class weights** nos modelos para compensar desbalanceamento sem perder dados reais.

### 2.4 Distribuições Numéricas e Outliers

Foram analisadas distribuições, estatísticas descritivas e outliers (via IQR) para idade, altura, peso, atividade física e consumo de água.

Principais conclusões:
- Idade entre 14 e 61 anos, bem distribuída.
- Altura e peso apresentam valores extremos biologicamente plausíveis.
- FAF (atividade física) e CH2O (consumo de água) mostram perfis bem distintos entre grupos, mas sem pontos obviamente errados.

Decisão: não remover outliers, pois representam possíveis casos reais, e os modelos baseados em árvore (Random Forest, Gradient Boosting) são robustos a esses extremos. A normalização com StandardScaler foi suficiente para os algoritmos sensíveis à escala.

### 2.5 Testes Estatísticos e Correlações

Foram aplicados testes de ANOVA (variáveis numéricas) e Qui-quadrado (variáveis categóricas), além de matriz de correlação, com foco em identificar fatores relevantes e possíveis colinearidades.

Achados principais:
- Variáveis fortemente associadas à obesidade: BMI (derivado de Weight e Height), Age, FAF (atividade física), FCVC (consumo de vegetais), FAVC (alimentos muito calóricos) e family_history.
- Correlação forte esperada entre Weight, Height e BMI, caracterizando circularidade com a definição de IMC.
- Correlações moderadas entre hábitos (por exemplo, maior atividade física associada a menor peso; maior consumo de vegetais associado a menor consumo de calóricos).

Decisões decorrentes:
- Criar BMI como feature derivada, mantendo Weight e Height para deixar o modelo decidir a importância relativa.
- Usar esses resultados para interpretar a importância de variáveis do modelo e embasar recomendações comportamentais.

### 2.6 Descoberta 5: Correlações Entre Variáveis

**Por que analisar correlações:**
- Identificar **multicolinearidade** (variáveis redundantes)
- Evitar overfitting
- Entender relações causais vs coincidências

**Correlações Fortes Encontradas (|r| > 0.5):**
- Weight ↔ Height: r = 0.67 (esperado: pessoas maiores pesam mais)
- Age ↔ Family History: r = 0.42 (confusão geracional)

**Correlações Moderadas (0.3 < |r| < 0.5):**
- FAF ↔ Weight: r = -0.38 (atividade física reduz peso)
- FCVC ↔ FAVC: r = -0.33 (quem come vegetais evita calóricos)

**Decisão Tomada:**
- Criar **BMI como feature derivada** (combina Weight e Height)
- Manter variáveis originais (modelos de árvore lidam bem com colinearidade)
- Monitorar importância de features após treinamento

### 2.7 Descoberta 6: Insights Comportamentais (Modificáveis)

**Por que focar em comportamentos modificáveis:**
- Genética não pode ser mudada (family_history, gender)
- Comportamentos podem ser alvo de **intervenções clínicas**
- Impacto prático maior na prevenção

#### Atividade Física (FAF)
**Descoberta:**
- Pessoas com obesidade tipo III: média 0.2 dias/semana
- Pessoas com peso normal: média 2.5 dias/semana
- **ANOVA:** F=450, p<0.0001 (diferença GIGANTE)

**Evidência Científica:**
- OMS recomenda ≥150 min/semana de atividade moderada
- Atividade física reduz risco de obesidade em 20-30%

**Implicação Prática:**
App pode recomendar: "Aumentar atividade física de X para Y dias/semana reduz risco em Z%"

#### Hábitos Alimentares (FAVC, FCVC)
**Descoberta:**
- 89% dos obesos tipo III consomem alimentos calóricos frequentemente
- Consumo de vegetais inversamente proporcional ao nível de obesidade

**Evidência Científica:**
- Alimentos ultraprocessados: +500 kcal/dia (Harvard, 2023)
- Dietas ricas em fibras/vegetais reduzem risco

**Implicação Prática:**
Dashboard pode identificar grupos de risco para intervenções nutricionais.

---

## 🤖 ETAPA 3: Estratégia de Modelagem

### 3.1 Visão Geral da Estratégia

Foram testados diferentes algoritmos de classificação multiclasse (Regressão Logística, Decision Tree, Random Forest, Gradient Boosting e XGBoost), todos inseridos no mesmo pipeline de pré-processamento.

Critérios de comparação:
- Acurácia (meta > 75%).
- F1-macro (para avaliar o equilíbrio entre classes, inclusive minoritárias).
- Matriz de confusão (para entender confusões entre níveis de obesidade).
- Tempo de treinamento e simplicidade de implantação.

O Random Forest se destacou como melhor compromisso entre desempenho, robustez e interpretabilidade (feature importance).

### 3.2 Pré-processamento e Engenharia de Atributos

Principais passos adotados (detalhados no notebook 02_model_training.ipynb):

- Conversão de variáveis categóricas em numéricas com LabelEncoder (uma codificação por coluna), reutilizada nos apps.
- Padronização de variáveis numéricas (Age, Height, Weight, BMI, FAF, CH2O, TUE) com StandardScaler para algoritmos sensíveis à escala.
- Criação da feature BMI = Weight / (Height²), mantida junto com Weight e Height.
- Divisão treino/teste estratificada (80/20), com random_state fixado para reprodutibilidade.

Esse pipeline foi salvo junto com o modelo para garantir consistência entre treinamento e uso em produção (Streamlit).

### 3.3 Validação e Comparação de Modelos

Foi utilizada validação com divisão treino/teste estratificada (80/20) e, no notebook 02_model_training.ipynb, validação cruzada para comparação entre algoritmos.

Pontos-chave:
- Proporção de classes preservada em treino e teste (stratify), garantindo avaliação justa em todas as classes.
- Random states fixados para permitir reprodutibilidade.
- Comparação de acurácia e F1-macro entre os modelos, antes de escolher o Random Forest otimizado como modelo final.

---

## 📈 ETAPA 4: Resultados de Modelagem (Resumo)

Os detalhes completos das métricas estão em 02_model_training.ipynb. Aqui, o foco é no que importa para o desafio e para o uso clínico.

### 4.1 Cenários Avaliados

Foram avaliados dois cenários principais:

1. **Modelo completo (com peso/altura/IMC)**  
   - Usa todas as variáveis, incluindo Weight, Height e BMI.  
   - Acurácia de teste próxima de 99%, bem acima da meta de 75%.  
   - Interpretação: o modelo essencialmente reaprende a regra de IMC, por isso a performance é muito alta.

2. **Modelo comportamental (sem peso/altura/IMC)**  
   - Remove variáveis antropométricas diretas (Weight, Height, BMI).  
   - Mantém variáveis demográficas e de hábitos (alimentação, atividade física, histórico familiar, etc.).  
   - Acurácia em torno de 87%, com F1-macro compatível, mostrando que hábitos e contexto já permitem uma boa triagem.

### 4.2 Métricas utilizadas

Para ambos os cenários foram analisadas:
- Acurácia.
- F1-macro.
- Matriz de confusão (para avaliar confusões entre níveis de obesidade).

Essas métricas são visualizadas e discutidas no notebook 02_model_training.ipynb, que complementa este guia.

### 4.2 Feature Importance (Importância das Variáveis)

**O que é:**
Ranking de quais features mais contribuem para predições do modelo.

**Por que é valioso:**
1. **Validação Científica:** Features importantes batem com literatura médica?
2. **Simplificação:** Podemos remover features irrelevantes?
3. **Interpretabilidade:** Explica decisões para médicos
4. **Intervenções:** Identifica alvos para mudanças comportamentais

**Expectativas (baseadas em EDA):**
- Top 3: BMI, family_history, FAF
- Médio: FAVC, FCVC, Age
- Baixo: SMOKE, CALC (poucos fumantes/bebedores no dataset)

---

## 🚀 ETAPA 5: Aplicação Prática (Streamlit Apps)

### 5.1 App de Predição Individual

**Objetivo:**
Profissional de saúde insere dados de um paciente → Sistema retorna nível de obesidade predito.

**Por que é útil:**
- Triagem rápida em consultórios
- Não requer cálculos manuais de IMC
- Considera múltiplos fatores além do peso

**Funcionalidades:**
1. **Formulário intuitivo** (dropdowns, sliders)
2. **Predição instantânea** com probabilidades
3. **Visualização gráfica** das probabilidades
4. **Recomendações contextualizadas** (baseadas em features importantes)

**Exemplo de Uso:**
```
Paciente: Mulher, 35 anos, 65kg, 1.60m
Histórico familiar: Sim
Atividade física: 1 dia/semana

→ Predição: Overweight Level I (78% de confiança)
→ Recomendação: Aumentar atividade física para ≥3 dias/semana
```

### 5.2 Dashboard Analítico

**Objetivo:**
Visão agregada de padrões populacionais para equipe médica ou gestores de saúde.

**Por que é útil:**
- Identifica grupos de risco em populações
- Monitora tendências ao longo do tempo
- Informa políticas de saúde pública

**Funcionalidades:**
1. **Filtros dinâmicos** (gênero, faixa etária, obesidade)
2. **Distribuições populacionais** (idade, IMC, comportamentos)
3. **Análises cruzadas** (obesidade × atividade física, obesidade × histórico familiar)
4. **Estatísticas descritivas** (médias, medianas por grupo)

**Exemplo de Insight:**
```
Homens 40-50 anos com histórico familiar:
- 65% estão em Obesity Type I ou superior
- Atividade física média: 0.8 dias/semana

→ Ação: Campanha de exercícios direcionada a este grupo
```

---

## 🎓 CONCLUSÕES E APRENDIZADOS (Resumo)

Principais conclusões que conectam dados, modelo e uso prático:

1. **Genética importa, mas não explica tudo**  
   - Histórico familiar tem associação forte com obesidade, alinhado com literatura sobre hereditariedade do IMC.  
   - Ainda assim, fatores comportamentais explicam boa parte da variação.

2. **Atividade física e alimentação são alvos prioritários**  
   - Diferenças claras de FAF e de padrão alimentar entre níveis de obesidade.  
   - Esses fatores são modificáveis e aparecem como importantes tanto na EDA quanto na importância de variáveis do modelo.

3. **Modelo completo x modelo comportamental**  
   - A alta acurácia do modelo com IMC é esperada pela circularidade.  
   - O resultado relevante para clínica é o desempenho do modelo comportamental (sem antropometria), que mantém boa capacidade de triagem.

4. **Aplicações em Streamlit tornam o resultado utilizável**  
   - O app de predição apoia decisões em nível individual.  
   - O dashboard facilita enxergar padrões populacionais e grupos de risco para intervenções.

### Decisões Metodológicas (Síntese)

Principais decisões tomadas ao longo do projeto, com justificativas de alto nível (detalhes adicionais em README.md e docs):

- Manter outliers plausíveis e utilizar modelos robustos (árvores) em vez de filtrar agressivamente.
- Não aplicar técnicas pesadas de balanceamento; usar class weights e avaliar F1-macro por classe.
- Criar BMI, mas também manter Weight e Height, permitindo que o modelo ajuste o peso relativo de cada variável.
- Testar múltiplos algoritmos e selecionar Random Forest otimizado com base em desempenho e robustez.
- Centralizar traduções e padrões visuais em código para facilitar entendimento por profissionais brasileiros.

### Limitações e Trabalhos Futuros

**Limitações Identificadas:**

1. **Generalização Geográfica:**
   - Dataset de México, Peru e Colômbia
   - Pode não generalizar para população brasileira ou europeia
   - Solução futura: Coletar dados locais para fine-tuning

2. **Causalidade vs Correlação:**
   - Modelos identificam associações, não causas
   - Exemplo: Baixa atividade física causa obesidade OU obesidade dificulta exercícios?
   - Solução futura: Estudos longitudinais

3. **Variáveis Ausentes:**
   - Não temos: renda, educação, acesso a alimentos, condições médicas
   - Podem ser confounders importantes
   - Solução futura: Dataset mais rico

4. **Progressão Temporal:**
   - Dados são cross-sectional (um ponto no tempo)
   - Não prevemos progressão de obesidade ao longo dos anos
   - Solução futura: Modelos de séries temporais

**Próximos Passos:**

1. **Validação Clínica:**
   - Testar predições com dados de hospitais brasileiros
   - Comparar com diagnósticos de endocrinologistas

2. **Explainability:**
   - Implementar SHAP values para explicar cada predição individual
   - Melhorar confiança de profissionais de saúde no sistema

3. **Mobile App:**
   - Versão para smartphones (automonitoramento)
   - Integração com wearables (atividade física automática)

4. **Intervenções Personalizadas:**
   - Sistema de recomendações baseado em perfil
   - Gamificação para engajamento em mudanças comportamentais

---

## 📚 Referências Acadêmicas

1. **Locke, A. E., et al. (2015).** Genetic studies of body mass index yield new insights for obesity biology. *Nature*, 518(7538), 197-206.
   - Base para importância de family_history

2. **World Health Organization (2020).** Physical Activity and Obesity Prevention.
   - Recomendações de atividade física (≥150 min/semana)

3. **Donnelly, J. E., et al. (2009).** Physical Activity Guidelines. *Medicine & Science in Sports & Exercise*.
   - Evidência de redução de 20-30% no risco de obesidade

4. **Harvard T.H. Chan School of Public Health (2023).** The Nutrition Source.
   - Impacto de ultraprocessados (+500 kcal/dia)

5. **Kanter, R., & Caballero, B. (2012).** Global gender disparities in obesity. *Obesity Reviews*.
   - Diferenças metabólicas e hormonais entre gêneros

6. **Claussnitzer, M., et al. (2015).** FTO Obesity Variant Circuitry. *New England Journal of Medicine*.
   - Mecanismos genéticos (FTO, MC4R, POMC)

---

## 🔗 Navegação do Projeto

**Próximos passos para avaliação:**

1. **01_exploratory_data_analysis.ipynb**
   - Código completo da análise exploratória
   - Visualizações principais
   - Testes estatísticos mais relevantes

2. **02_model_training.ipynb**
   - Pré-processamento de dados
   - Treinamento e comparação de modelos
   - Avaliação de desempenho e seleção do modelo final

3. **app/app_prediction.py**
   - Execute: `streamlit run app/app_prediction.py`
   - Teste predições individuais com diferentes perfis de paciente

4. **app/app_dashboard.py**
   - Execute: `streamlit run app/app_dashboard.py`
   - Explore análises populacionais e padrões de risco

5. **Documentação complementar:**
   - `README.md`: contexto, setup e visão geral do projeto
   - `docs/DOCUMENTACAO_TECNICA.md`: detalhes técnicos e padronizações


---

## ✅ Checklist de Avaliação

**Para o Avaliador:**

- [ ] **Compreensão do Problema:** Contexto de saúde pública claro?
- [ ] **Qualidade da EDA:** Análises relevantes e bem fundamentadas?
- [ ] **Rigor Estatístico:** Testes apropriados aplicados e interpretados?
- [ ] **Justificativa de Decisões:** Escolhas metodológicas explicadas?
- [ ] **Insights Acionáveis:** Descobertas traduzidas em recomendações práticas?
- [ ] **Acurácia do Modelo:** >75% alcançado? (verificar notebook 02)
- [ ] **Funcionalidade dos Apps:** Streamlit apps funcionando corretamente?
- [ ] **Profissionalismo:** Visualizações, código e documentação de qualidade?
- [ ] **Reprodutibilidade:** Código documentado, random states fixados?
- [ ] **Limitações Reconhecidas:** Consciência crítica das restrições?

---

**Última Atualização:** 18/12/2025  
**Contato:** POSTECH Data Analytics - Tech Challenge Fase 4